# Bounds in Probability

**Adapted from**: SOSTOOLS' SOSDEMO8 (See Section 4.8 of [SOSTOOLS User's Manual](http://sysos.eng.ox.ac.uk/sostools/sostools.pdf))

The probability adds up to one.

In [1]:
μ0 = 1

1

The mean is one.

In [2]:
μ1  = 1

1

The standard deviation is 1/2.

In [3]:
σ = 1/2

0.5

The second moment `E(x^2)` is:

In [4]:
μ2 = σ^2 + μ1^2

1.25

We define the moments as follows:

In [5]:
using DynamicPolynomials
@polyvar x
monos = [1, x, x^2]
using SumOfSquares
μ = measure([μ0, μ1, μ2], monos)

MultivariateMoments.Measure{Float64, DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}([1.25, 1.0, 1.0], DynamicPolynomials.Monomial{true}[x², x, 1])

We need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v0.21.6/installation/#Supported-solvers) for a list of the available choices.
We use `SOSModel` instead of `Model` to be able to use the `>=` syntax for Sum-of-Squares constraints.

In [6]:
using CSDP
solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
model = SOSModel(solver);

We create a polynomial with the monomials in `monos` and
JuMP decision variables as coefficients as follows:

In [7]:
@variable(model, poly, Poly(monos))

(noname)x² + (noname)x + (noname)

Nonnegative on the support:

In [8]:
K = @set 0 <= x && x <= 5
con_ref = @constraint(model, poly >= 0, domain = K)

(noname)x² + (noname)x + (noname) is SOS

Greater than one on the event:

In [9]:
@constraint(model, poly >= 1, domain = (@set 4 <= x && x <= 5))

(noname)x² + (noname)x + (noname - 1) is SOS

The bound (we use `LinearAlgebra` for the `⋅` syntax for the scalar product):

In [10]:
using LinearAlgebra
@objective(model, Min, poly ⋅ μ)

1.25 noname + noname + noname

We verify that we found a feasible solution:

In [11]:
optimize!(model)
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

The objective value is `1/37`:

In [12]:
objective_value(model)

0.02702702756150918

The solution is `(12x-11)^2 / 37^2`:

In [13]:
value(poly) * 37^2

143.9999815881086x² - 263.9999061090176x + 120.99992985558798

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*